In [0]:
%run ../configurations/paths_config

In [0]:
dbutils.widgets.text("p_file_date","2021-03-21")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
from pyspark.sql.types import StringType,IntegerType,DoubleType,TimestampType,DateType,StructType,StructField
from pyspark.sql.functions import col,lit
from pyspark.sql import functions as sf

In [0]:
new_schema = StructType(fields=[StructField("constructorId",IntegerType(),False),
                                StructField("constructorRef",StringType(),True),
                                StructField("name",StringType(),True),
                                StructField("nationality",StringType(),True),
                                StructField("url",StringType(),True)])

In [0]:
raw_df = spark.read.schema(new_schema).json(raw_path+f"/{v_file_date}/constructors.json")

In [0]:
trimmed_df = raw_df.drop("url")

In [0]:
renamed_df = trimmed_df.withColumnRenamed("constructorId","constructor_id").withColumnRenamed("constructorRef","constructor_ref")

In [0]:
audited_df = renamed_df.withColumn("ingestion_timestamp",sf.current_timestamp())

In [0]:
audited_df.write.mode("overwrite").format("delta").saveAsTable("formula1_silver.constructors")